In [5]:
from fastai.tabular.all import *
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')
train_data.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [31]:
splits = RandomSplitter(valid_pct=0.2)(train_data)
y_names = ["Pastry", "Z_Scratch", "K_Scatch", "Stains", "Dirtiness", "Bumps", "Other_Faults"]

In [38]:
dls = TabularPandas(train_data, splits=splits, procs=[FillMissing, Normalize, Categorify], y_names=y_names, y_block=MultiCategoryBlock(encoded=True, vocab=y_names),
                    cat_names=["id"],
                    cont_names=['X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum', 'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas', 'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index', 'SigmoidOfAreas'],
                    ).dataloaders()

In [39]:
dls.show_batch()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,12179,1456.999974,1470.000003,5.897400e+05,5.897520e+05,73.999975,24.000002,11.999999,6480.994670,79.000000,101.0,1630.000008,1.000000e+00,2.658388e-08,100.000001,0.2630,0.3392,0.3333,0.0155,0.9583,0.7500,-1.971944e-08,1.8692,1.3222,0.7782,-0.6667,-0.3041,0.2116,False,False,False,False,False,True,False
1,501,1494.999998,1505.999994,2.837245e+06,2.837268e+06,196.000021,23.000002,22.000001,18326.996268,77.000000,110.0,1685.999997,1.000000e+00,2.658388e-08,60.000000,0.2188,0.2597,0.7778,0.0083,0.6087,1.0000,1.000000e+00,2.2989,1.1461,1.2553,0.2222,-0.2651,0.4170,True,False,False,False,False,False,False
2,2234,1603.000000,1614.000027,7.071220e+05,7.071370e+05,514.999989,15.000001,49.000000,41622.002489,49.000000,84.0,1687.000008,8.007827e-09,1.000000e+00,149.999997,0.0012,0.4463,0.2857,0.0071,0.8000,1.0000,1.000000e+00,2.7118,1.0792,1.6902,0.7143,-0.4550,0.9216,False,False,False,False,False,False,True
3,10816,1119.999995,1144.999989,2.378173e+06,2.378236e+06,1183.000022,106.000000,87.999999,51660.994216,103.000000,127.0,1354.000002,8.007827e-09,1.000000e+00,174.999999,0.2278,0.4771,0.4839,0.0081,0.2903,1.0000,1.000000e+00,3.0516,1.0414,1.9445,0.5161,-0.1063,0.9999,True,False,False,False,False,False,False
4,9786,1196.000012,1204.999994,2.519944e+06,2.519955e+06,137.000022,15.000001,11.999999,18012.996683,116.000002,140.0,1360.000000,8.007827e-09,1.000000e+00,50.000001,0.2127,0.2159,0.7727,0.0088,0.8000,0.9167,-1.971944e-08,2.1367,1.0792,1.0414,-0.2273,0.0026,0.3329,False,False,False,True,False,False,False
5,3972,40.999999,213.000020,1.821207e+06,1.821379e+06,6602.999833,705.000013,419.999989,684731.004313,38.000001,124.0,1402.000001,8.007827e-09,1.000000e+00,40.000000,0.0585,0.5054,0.9576,0.1268,0.2772,0.3857,-1.971944e-08,3.8197,2.2355,2.2095,-0.0424,-0.1924,1.0000,False,False,False,False,False,False,True
6,2203,40.999999,213.000020,2.740809e+06,2.740889e+06,6276.999860,285.999995,150.999998,639432.996333,40.000000,124.0,1358.000003,8.007827e-09,1.000000e+00,40.000000,0.0604,0.4562,0.3931,0.1318,0.6279,0.5402,-1.971944e-08,3.7977,2.2553,1.8325,-0.6069,-0.1951,1.0000,False,False,True,False,False,False,False
7,18576,805.000000,811.000000,1.341292e+06,1.341296e+06,15.999961,7.000000,4.000003,2034.002720,110.000001,141.0,1360.000000,8.007827e-09,1.000000e+00,50.000001,0.7393,0.2000,0.8000,0.0037,0.7143,1.0000,-1.971944e-08,1.2041,0.6990,0.6021,-0.2000,-0.0208,0.1322,False,False,False,True,False,False,False
8,11493,40.999999,211.000014,1.447908e+06,1.447954e+06,6276.999860,262.999996,134.999999,659057.983473,42.000002,124.0,1355.999998,8.007827e-09,1.000000e+00,40.000000,0.0605,0.4495,0.3846,0.1252,0.6364,0.4667,-1.971944e-08,3.7977,2.2430,1.8129,-0.6154,-0.1851,1.0000,False,False,True,False,False,False,False
9,10368,1072.999992,1094.000011,7.430055e+03,7.457952e+03,244.999952,27.999999,28.000001,23641.997662,79.000000,103.0,1691.999995,1.000000e+00,2.658388e-08,300.000000,0.4275,0.5831,0.9643,0.0164,0.9643,0.9643,-1.971944e-08,2.3856,1.4314,1.4314,-0.0357,-0.3307,0.7436,False,False,False,False,False,False,True


In [40]:
learn = tabular_learner(dls, metrics=accuracy_multi)

In [41]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy_multi,time
0,0.513034,0.403880,0.867886,00:10
1,0.162537,0.263055,0.873982,00:09
2,0.057988,0.256224,0.883201,00:10
3,0.014771,0.278434,0.887179,00:10
4,0.005440,0.282333,0.887328,00:10


In [42]:
learn.show_results()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,Pastry_pred,Z_Scratch_pred,K_Scatch_pred,Stains_pred,Dirtiness_pred,Bumps_pred,Other_Faults_pred
0,0.0,1.836936,1.883647,-0.941575,-0.944877,-0.356549,-0.454342,-0.247442,-0.343232,-1.736122,-3.129427,1.580647,-0.824663,0.826334,4.119809,-1.101150,0.168001,-1.620339,-0.560065,-0.835946,0.761252,0.842736,0.162787,-0.999646,0.496734,1.520815,-3.057794,-0.949887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,-1.231884,-1.359461,0.102700,0.104802,-0.244248,-0.241453,0.229892,-0.281003,-1.248844,-0.184032,-0.458180,-0.824663,0.826334,0.800994,-0.920207,-0.456098,-1.504508,-0.272771,-1.033897,0.710930,0.842736,0.563232,0.147909,1.360017,1.459334,-0.385187,1.291991,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.603623,1.655569,0.099046,0.101089,-0.451963,-0.375909,-0.535831,-0.420471,0.352212,-0.113904,1.279612,1.212615,-1.210164,0.063480,0.746469,-1.893785,-0.284036,-0.364279,1.734714,-0.598813,-1.231591,-0.957719,0.291595,-1.351742,-1.237837,0.213146,-1.069675,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.489741,0.451157,0.681052,0.686087,-0.445208,-0.459945,-0.515943,-0.411804,0.491434,0.306867,1.580647,1.212615,-1.210164,-0.120899,1.507493,-1.652692,1.324857,-0.508991,1.042336,0.761252,0.842736,-0.753138,-0.578668,-0.771677,-0.042441,0.586067,-0.991526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.681660,0.679235,-0.226861,-0.226494,-0.366682,-0.375909,-0.436387,-0.348333,0.804684,0.376995,-0.718165,-0.824663,0.826334,-0.674035,-1.067653,-1.151155,0.874274,-0.396201,0.128164,0.761252,0.842736,0.107977,-0.229748,-0.222090,0.196720,0.420324,0.184630,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-1.333488,-1.477501,1.851939,-0.944745,-0.453652,-0.353500,-0.386665,-0.419518,0.700267,0.236738,1.566964,1.212615,-1.210164,1.354130,-0.203637,1.096087,-0.498709,-0.521759,-1.077086,-0.145458,0.842736,-1.022699,-0.667453,0.018013,0.925474,0.363143,-1.166532,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,-0.129292,-0.193063,-0.327019,-0.327156,-0.414248,-0.387114,-0.307109,-0.395591,-0.274288,-1.937244,1.594330,1.212615,-1.210164,0.063480,1.264253,-0.510123,-1.218019,-0.447275,1.734714,0.486519,0.842736,-0.257072,-0.423188,0.249692,1.307271,-1.413623,-0.486425,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.153931,1.157399,0.276741,0.279724,-0.282245,-0.191031,0.269670,-0.296315,0.143378,0.797766,-0.718165,-0.824663,0.826334,-0.674035,-0.838814,0.425220,-1.843121,-0.489838,-1.364564,0.248054,0.842736,0.437763,-0.667453,1.372406,1.639268,-0.383530,1.261818,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,-0.229014,-0.307102,0.334442,0.337695,-0.448304,-0.487956,-0.525887,-0.416909,0.352212,-0.674932,1.525913,1.212615,-1.210164,-0.120899,1.547563,-1.652692,0.593577,-0.547297,1.734714,0.761252,0.842736,-0.837929,-0.875261,-0.865340,0.345708,-0.176351,-1.141790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [43]:
test_data = pd.read_csv('test.csv')
test_dl = learn.dls.test_dl(test_data)
preds, _ = learn.get_preds(dl=test_dl)
preds

tensor([[3.9490e-01, 4.7725e-03, 4.2284e-03,  ..., 2.1072e-02, 5.9182e-02,
         1.7950e-01],
        [3.0753e-02, 5.8403e-03, 1.3382e-03,  ..., 1.2388e-02, 3.6014e-02,
         3.5022e-01],
        [3.2487e-03, 2.3059e-03, 4.0079e-03,  ..., 3.1249e-03, 8.7696e-02,
         6.8782e-01],
        ...,
        [2.1827e-03, 1.2869e-03, 9.9238e-01,  ..., 9.8295e-04, 2.1267e-03,
         2.5795e-02],
        [2.1314e-01, 5.3678e-03, 4.1738e-03,  ..., 1.4441e-02, 6.4701e-02,
         2.4618e-01],
        [3.5869e-03, 2.0153e-03, 9.8626e-01,  ..., 1.3010e-03, 2.9879e-03,
         1.0326e-02]])

In [44]:
binary_preds = (preds > 0.5).int()
binary_preds


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        ...,
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0]], dtype=torch.int32)